Выполнила: Анастасяи Плавина

## Тема 2. Бейзлайны и детерминированные алгоритмы item-item

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
result = pd.read_csv('predictions_basic.csv')
result

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[855282, 12754926, 10255555, 10459882, 999360]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[8352923, 10254454, 126637, 10282033, 1124719]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[1071342, 9932329, 985893, 926549, 7186250]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[1082185, 1029743, 6534178, 1127831, 995785]"
3,7,[ 840386 889774 898068 909714 929067 ...,"[8181185, 900491, 921622, 1317910, 944599]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[1082185, 1029743, 1127831, 995785, 1044078]"
4,8,[ 835098 872137 910439 924610 992977 ...,"[1034655, 928892, 950441, 925445, 991951]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 1029743, 1098066, 6534178, 1127831]"
...,...,...,...,...,...,...,...,...
2037,2496,[6534178],"[1406777, 16769463, 964450, 10457180, 10255413]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 1098066, 999999, 826249]","[1082185, 1098066, 6534178, 995785, 1068719]"
2038,2497,[ 1016709 9835695 1132298 16809501 845294 ...,"[1101560, 15660208, 1079365, 1259979, 2456053]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 1127831]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 1098066, 999999, 826249]","[1082185, 1029743, 1098066, 6534178, 1127831]"
2039,2498,[15716530 834484 901776 914190 958382 ...,"[834572, 1036057, 1094203, 5564670, 87243]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 1127831]","[1082185, 999999, 981760, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 1029743, 1068719, 1106523, 5569230]"
2040,2499,[ 867188 877580 902396 914190 951590 ...,"[2400179, 330850, 1079099, 7168859, 551900]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 1098066, 999999, 826249]","[1082185, 1098066, 6534178, 1004906, 1081177]"


In [5]:
def from_str_to_array(row):
    """converting array strings to lists
    """
    row['actual'] = [int(x) for x in row['actual'][1:-1].split()]
    for col in result.columns[2:]:

        if row[col] != '[]':
            row[col] = [int(x) for x in row[col][1:-1].split(', ')]
        else:
            row[col] = []
    return row

result = result.apply(from_str_to_array, axis=1)
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[855282, 12754926, 10255555, 10459882, 999360]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8352923, 10254454, 126637, 10282033, 1124719]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [6]:
# создаем массив весов
items_weights = data_train.groupby('item_id')['quantity'].sum().reset_index()
items_weights.rename(columns={'quantity': 'n_sold'}, inplace=True)
items_weights['weight'] = items_weights['n_sold'] / items_weights['n_sold'].sum()
items_weights = items_weights[['item_id', 'weight']]
items_weights

,item_id,weight
0,25671,2.617171e-08
1,26081,4.361952e-09
2,26093,4.361952e-09
3,26190,4.361952e-09
4,26355,8.723905e-09
...,...,...
86860,17381856,0.000000e+00
86861,17382205,4.361952e-09
86862,17383227,4.361952e-09
86863,17827644,4.361952e-09


In [7]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации c весами
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights['item_id'])
    items_weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights)
    
    return recs.tolist()

In [8]:
%%time

items = data_train.item_id.unique()

result['weighted_random_recommendation'] = result['user_id'] \
                    .apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 4.83 s, sys: 180 ms, total: 5.01 s
Wall time: 5.05 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[855282, 12754926, 10255555, 10459882, 999360]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[6534166, 6534178, 6544236, 6533889, 5747420]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8352923, 10254454, 126637, 10282033, 1124719]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[6534178, 6533889, 6534166, 397896, 6544236]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [9]:
def precision_at_k(recommended_list, actual_list, k=5):
    """calculating precision@k"""

    actual_list = np.array(actual_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(actual_list, recommended_list)
    if recommended_list.size:
        return flags.sum() / recommended_list.size
    else:
        return 0

In [10]:
for name_col in result.columns[2:]:
    print(f"{result.apply(lambda row: precision_at_k(row[name_col], row['actual']), 1).mean():.4f}:{name_col}")

0.0005:random_recommendation
0.1552:popular_recommendation
0.1369:itemitem
0.1329:cosine
0.1390:tfidf
0.2011:own_purchases
0.0477:weighted_random_recommendation


__Вывод:__ Анализ метрик показал, что случайные рекомендации с учетом весов на зеачительно более эффективны, однако все еще не очень успешны по сравнению с другими методами рекомендаций. Наиболее оптимальный бейзлайн среди текущих = предсказание на основе уже купленных товаров

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [11]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_1000 = popularity.sort_values('n_sold', ascending=False).head(1000).item_id.tolist()
data_train.loc[~data_train['item_id'].isin(top_1000), 'item_id'] = 999999

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


#### Random

In [12]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""

    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [13]:
%%time

items = data_test.item_id.unique()

result['random_recommendation_top_n'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

# result.head(2)

CPU times: user 1.09 s, sys: 12.1 ms, total: 1.1 s
Wall time: 1.11 s


#### Popularity

In [14]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [15]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation_top_n'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 40.5 ms, sys: 33.8 ms, total: 74.3 ms
Wall time: 73.3 ms


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_top_n,popular_recommendation_top_n
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[855282, 12754926, 10255555, 10459882, 999360]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[6534166, 6534178, 6544236, 6533889, 5747420]","[929178, 9836933, 1118244, 10121834, 962708]","[999999, 6534178, 6533889, 1029743, 6534166]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8352923, 10254454, 126637, 10282033, 1124719]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[6534178, 6533889, 6534166, 397896, 6544236]","[1078912, 839419, 1043563, 931842, 1059153]","[999999, 6534178, 6533889, 1029743, 6534166]"


#### Weigted random recommendation

In [16]:
%%time

items = data_train.item_id.unique()

result['weighted_random_recommendation_top_n'] = result['user_id'] \
                    .apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 4.79 s, sys: 55.3 ms, total: 4.84 s
Wall time: 4.88 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_top_n,popular_recommendation_top_n,weighted_random_recommendation_top_n
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[855282, 12754926, 10255555, 10459882, 999360]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[6534166, 6534178, 6544236, 6533889, 5747420]","[929178, 9836933, 1118244, 10121834, 962708]","[999999, 6534178, 6533889, 1029743, 6534166]","[6534166, 6534178, 6533889, 480014, 1404121]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8352923, 10254454, 126637, 10282033, 1124719]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[6534178, 6533889, 6534166, 397896, 6544236]","[1078912, 839419, 1043563, 931842, 1059153]","[999999, 6534178, 6533889, 1029743, 6534166]","[6534178, 6534166, 1404121, 707683, 6533889]"


#### Item-item

In [17]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit


# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [18]:
user_item_matrix.shape

(2499, 1001)

In [19]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

11.90494179689858

In [20]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [21]:
model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['itemitem_top_n'] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

#### Cosine 

In [22]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


CPU times: user 372 ms, sys: 20.6 ms, total: 393 ms
Wall time: 244 ms


In [23]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 999999, 981760, 1127831, 961554]

In [24]:
%%time

result['cosine_top_n'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 66.5 ms, sys: 5 ms, total: 71.5 ms
Wall time: 70.8 ms


In [25]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


CPU times: user 390 ms, sys: 16.5 ms, total: 406 ms
Wall time: 250 ms


In [26]:
[id_to_itemid[rec[0]] for rec in recs]

[1082185, 981760, 999999, 1127831, 961554]

In [27]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

CPU times: user 76 ms, sys: 5.07 ms, total: 81.1 ms
Wall time: 83.2 ms


In [28]:
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_top_n,popular_recommendation_top_n,weighted_random_recommendation_top_n,itemitem_top_n,cosine_top_n
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[855282, 12754926, 10255555, 10459882, 999360]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]","[6534166, 6534178, 6544236, 6533889, 5747420]","[929178, 9836933, 1118244, 10121834, 962708]","[999999, 6534178, 6533889, 1029743, 6534166]","[6534166, 6534178, 6533889, 480014, 1404121]","[999999, 1082185, 981760, 995242, 1127831]","[1082185, 999999, 981760, 1127831, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8352923, 10254454, 126637, 10282033, 1124719]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[6534178, 6533889, 6534166, 397896, 6544236]","[1078912, 839419, 1043563, 931842, 1059153]","[999999, 6534178, 6533889, 1029743, 6534166]","[6534178, 6534166, 1404121, 707683, 6533889]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 999999, 981760, 1098066, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1071342, 9932329, 985893, 926549, 7186250]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1127831, 878996]","[1082185, 1029743, 6534178, 1127831, 995785]","[6534178, 6533889, 6410462, 480014, 5668996]","[14025127, 13006983, 13213453, 1022254, 15716479]","[999999, 6534178, 6533889, 1029743, 6534166]","[6534178, 6533889, 6544236, 909894, 6534166]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[8181185, 900491, 921622, 1317910, 944599]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 1029743, 1127831, 995785, 1044078]","[6534178, 6533889, 6534166, 480014, 5703832]","[871264, 9297571, 1069094, 1137010, 1024384]","[999999, 6534178, 6533889, 1029743, 6534166]","[6534178, 6534166, 6533889, 1426702, 1007195]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1034655, 928892, 950441, 925445, 991951]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 1029743, 1098066, 6534178, 1127831]","[6534178, 6533889, 1388206, 6534166, 5703832]","[845269, 1025628, 1021486, 835578, 990797]","[999999, 6534178, 6533889, 1029743, 6534166]","[6534178, 940139, 1426702, 6544236, 6533889]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 999999, 1098066, 826249]"


In [29]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


CPU times: user 387 ms, sys: 18.7 ms, total: 405 ms
Wall time: 270 ms


In [30]:
# print map item_id to row_id
itemid_to_id
print(*[f"Item ID {id_to_itemid[rec[0]]} --> Row ID {rec[0]}" for rec in recs],sep='\n')

Item ID 999999 --> Row ID 524
Item ID 1082185 --> Row ID 720
Item ID 1029743 --> Row ID 590
Item ID 1004906 --> Row ID 534
Item ID 1081177 --> Row ID 718


In [31]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)])

CPU times: user 63.7 ms, sys: 3.69 ms, total: 67.4 ms
Wall time: 65.1 ms


In [32]:
%%time

model = ItemItemRecommender(K=2, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

result['own_purchases_top_n'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=False)])


CPU times: user 417 ms, sys: 24 ms, total: 441 ms
Wall time: 304 ms


In [33]:
results_metrics = {'method':[], 
                  'precision@k': []}

for name_col in result.columns[2:]:
#     print(f"{result.apply(lambda row: precision_at_k(row[name_col], row['actual']), 1).mean():.4f}:{name_col}")
    results_metrics['precision@k'].append(result \
                                    .apply(lambda row: precision_at_k(row[name_col], row['actual']), 1).mean())
    results_metrics['method'].append(name_col)

In [34]:
df = pd.DataFrame(results_metrics)
df = df.sort_values('precision@k', ascending=False)
df

,method,precision@k
12,own_purchases_top_n,0.219434
5,own_purchases,0.196074
1,popular_recommendation,0.155240
10,itemitem_top_n,0.140353
4,tfidf,0.139765
2,itemitem,0.136925
11,cosine_top_n,0.133692
3,cosine,0.132909
8,popular_recommendation_top_n,0.085602
6,weighted_random_recommendation,0.047698


Лучший бейзлайн по precision@k получили на выборке top-1000 товаров, при этом на основе купленных товаров. Сужение выборки товаров также дало улучшение item-item.